<a href="https://www.kaggle.com/code/drkaggle22/toxic-comments-classification-tf-idf-nltk?scriptVersionId=176832862" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_320.npy
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
df_train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip', index_col='id')

df_test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip', index_col='id')

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df_train['comment_text'] = df_train['comment_text'].str.lower()
df_test['comment_text'] = df_test['comment_text'].str.lower()

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153164 entries, 00001cee341fdb12 to ffffce3fb183ee80
Data columns (total 1 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_text  153164 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   toxic          159571 non-null  int64 
 2   severe_toxic   159571 non-null  int64 
 3   obscene        159571 non-null  int64 
 4   threat         159571 non-null  int64 
 5   insult         159571 non-null  int64 
 6   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 9.7+ MB


In [7]:
import re

def remove_special_characters(text):
    text = re.sub(r'http\S+', ' ', text )
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\bhttps?://[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\d', ' ', text)  # Corrected line
    text= re.sub(r'[\u4e00-\u9fff]+', ' ', text)
    return text

df_train['comment_text'] = df_train['comment_text'].apply(remove_special_characters)
df_test['comment_text'] = df_test['comment_text'].apply(remove_special_characters)

In [8]:
file_path = '/kaggle/input/toxic-comments-transformer-embeddings/train_embs_400.npy'
df_train_emb = np.load(file_path)

In [9]:
(df_train_emb).shape

(159571, 768)

In [10]:
X = df_train_emb  # Assuming 'comment_text' contains your text data
y = df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

In [11]:
from sklearn.model_selection import train_test_split
# Splitting the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=42)

In [12]:
x = X_train
val_x = X_valid

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer


# Initialize MultiLabelBinarizer with the classes
mlb = MultiLabelBinarizer()

# Transform target labels into binary form
y_train_binary = mlb.fit_transform(y_train)
y_valid_binary = mlb.transform(y_valid)

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [14]:
import numpy as np

# List of class labels
class_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Initialize empty arrays to store binary labels
y_train_binary = np.zeros((len(y_train), len(class_labels)), dtype=int)
y_valid_binary = np.zeros((len(y_valid), len(class_labels)), dtype=int)

# Manually create binary labels for each class
for i, label in enumerate(class_labels):
    y_train_binary[:, i] = y_train[label]
    y_valid_binary[:, i] = y_valid[label]

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [15]:
print("y_train_binary shape:", y_train_binary.shape)
print("y_valid_binary shape:", y_valid_binary.shape)


y_train_binary shape: (127656, 6)
y_valid_binary shape: (31915, 6)


In [16]:
from sklearn.neural_network import MLPClassifier
import numpy as np

epochs = 50  # Define the number of epochs
# Set up the MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(64, 32, 10, 6),
                    activation='relu',
                    solver='adam',
                    random_state=42,
                    verbose=True,
                    learning_rate_init=0.001,
                    max_iter=epochs,  
                    early_stopping=True,  # Enable early stopping
                    n_iter_no_change=5,    # Number of iterations with no improvement to wait before stopping
                    )

# Fit the classifier with the training data and binary target labels
clf.fit(X_train, y_train_binary)


Iteration 1, loss = 1.09742092
Validation score: 0.902162
Iteration 2, loss = 0.39892114
Validation score: 0.908037
Iteration 3, loss = 0.36423572
Validation score: 0.908664
Iteration 4, loss = 0.34754991
Validation score: 0.911170
Iteration 5, loss = 0.34019824
Validation score: 0.910465
Iteration 6, loss = 0.33566307
Validation score: 0.912580
Iteration 7, loss = 0.33172777
Validation score: 0.910779
Iteration 8, loss = 0.32794959
Validation score: 0.913285
Iteration 9, loss = 0.32601110
Validation score: 0.914617
Iteration 10, loss = 0.32444225
Validation score: 0.914460
Iteration 11, loss = 0.32193470
Validation score: 0.907959
Iteration 12, loss = 0.31979097
Validation score: 0.912189
Iteration 13, loss = 0.31779788
Validation score: 0.915400
Iteration 14, loss = 0.31783529
Validation score: 0.913050
Iteration 15, loss = 0.31343044
Validation score: 0.912659
Iteration 16, loss = 0.31371966
Validation score: 0.911405
Iteration 17, loss = 0.31125648
Validation score: 0.911249
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(64, 32, 10, 6),
              max_iter=50, n_iter_no_change=5, random_state=42, verbose=True)

In [17]:
file_path_2 = '/kaggle/input/toxic-comments-transformer-embeddings/test_embs_400.npy'

df_test_emb = np.load(file_path_2)

In [18]:
df_test_emb.shape

(153164, 768)

In [19]:
# Assuming clf is the trained MLPClassifier model

# Predict probabilities for test data
y_test_prob = clf.predict_proba(df_test_emb)

# Print the shape of predicted probabilities for confirmation
print("Shape of predicted probabilities:", y_test_prob.shape)

# Now you have predicted probabilities for each class for the test data


Shape of predicted probabilities: (153164, 6)


In [20]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [21]:
# Convert the predicted probabilities to float
y_test_prob_float = y_test_prob.astype(float)

# Print the updated type of predicted probabilities for confirmation
print("Type of predicted probabilities:", y_test_prob_float.dtype)

# Now y_test_prob_float contains predicted probabilities for each class for each data point in the test data, stored as float numbers


Type of predicted probabilities: float64


In [22]:
import numpy as np

# Assuming y_test_prob_float is your array of predicted probabilities

# Set numpy printing options to suppress scientific notation
np.set_printoptions(suppress=True)

# Print the updated predicted probabilities
print(y_test_prob_float)


[[0.98186481 0.29842758 0.88437897 0.07291752 0.83523506 0.17722271]
 [0.00013484 0.00000001 0.00003191 0.00000023 0.00001623 0.00000541]
 [0.00067263 0.00000018 0.00018899 0.0000031  0.00012628 0.00004571]
 ...
 [0.00023587 0.00000002 0.00005987 0.00000055 0.00003271 0.00001141]
 [0.00044821 0.00000007 0.00012003 0.00000162 0.00007562 0.0000266 ]
 [0.82689309 0.02191269 0.4234373  0.01288718 0.3787654  0.0416129 ]]


In [23]:
# Set the floating-point format for display
pd.options.display.float_format = '{:.10f}'.format

# Create submission DataFrame
submid = pd.DataFrame({'id': df_test.index})  # Use index as 'id' column
submission = pd.concat([submid, pd.DataFrame(y_test_prob_float, columns=classes)], axis=1)

# Write submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)


In [24]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.9818648100,0.2984275818,0.8843789697,0.0729175210,0.8352350593,0.1772227138
1,0000247867823ef7,0.0001348410,0.0000000054,0.0000319079,0.0000002272,0.0000162297,0.0000054082
2,00013b17ad220c46,0.0006726325,0.0000001782,0.0001889900,0.0000031002,0.0001262789,0.0000457061
3,00017563c3f7919a,0.0000576899,0.0000000009,0.0000129188,0.0000000517,0.0000052564,0.0000017787
4,00017695ad8997eb,0.0003091968,0.0000000298,0.0000761716,0.0000010044,0.0000496963,0.0000159441
